# Carga datos estudio índices climáticos
## 1. Carga datos PTI-grid-v0 (res 2.5 km).
## 2. Carga datos ERA5-Land (res 10 km). 
## 3. Interpolo datos PTI-grid-v0 a ERA5-Land.
## Pablo Lavín

In [1]:
options(java.parameters = "-Xmx32g")
library(rJava)

In [2]:
# Cargamos paquetes

library(repr)
library(dplyr)

library(abind)
library(loadeR)
library(transformeR)
library(convertR)
library(visualizeR)
library(downscaleR)
library(climate4R.UDG)
library(climate4R.climdex)
library(climate4R.indices)
library(easyVerification)

library(lattice)
library(magrittr)
library(gridExtra)
library(RColorBrewer)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: loadeR.java

Java version 23x amd64 by N/A detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.6 (2023-06-26) is loaded

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.8.1 (2023-06-22) is loaded


Get the latest stable version (1.8.2) using <devtools::install_github(c('SantanderMetGroup/climate4R.UDG','SantanderMetGroup/loadeR'))>

Please use 'citation("loadeR")' to cite this package.




    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded


Get the latest stable version (2.2.3) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.

Loading required package: udunits2

udunits system database read from /vols/abedul/home/meteo/lavinp/miniforge3/envs/C4R/share/udunits/udunits2.xml

convertR version 0.3.0 (2025-07-31) is loaded


Development version may have an unexpected behaviour

  More information about the 'climate4R' ecosystem in: http://meteo.unican.es/climate4R


Attaching package: ‘convertR’


The following objects are masked from ‘package:loadeR’:

    hurs2huss, huss2hurs, tdps2hurs


visualizeR version 1.6.4 (2023-10-26) is loaded

Please see 'citation("visualizeR")' to cite this package.

downscaleR version 3.3.4 (2023-06-22) is loaded

Please use 'citation("downscaleR")' to cite this package.

Loading required package: climdex.pcic

Loading required package: PCICt

climate4R.climdex version 0

In [3]:
# Region de estudio

lon = c(-10, 5)
lat = c(35,44)

# Color
color = colorRampPalette(rev(brewer.pal(n = 9, "RdYlBu")))

## 1. Carga datos PTI-grid-v0 (res 2.5 km)

In [ ]:
## Cargo datos de temperatura media y mínima, humedad relativa y precipitación

# Rutas
ruta_tmax = "/lustre/gmeteo/PTICLIMA/DATA/OBSERVATIONS/PTI-grid-v0/data/Iberia/day/tmax_daily_grid_pen.nc"
ruta_tmin = "/lustre/gmeteo/PTICLIMA/DATA/OBSERVATIONS/PTI-grid-v0/data/Iberia/day/tmin_daily_grid_pen.nc"

meses_num = 1:12
meses_str = sprintf("%02d", meses_num)

for (m in seq_along(meses_num)) {
    m_num = meses_num[m]
    m_str = meses_str[m]

    # Cargar tmax del mes m
    tmax_mes = loadGridData(
        dataset = ruta_tmax,
        var = "tmax",
        latLim = lat,
        lonLim = lon,
        years = 1981:2021,
        season = m_num) %>% suppressMessages() %>% suppressWarnings()
    
    # Cargar tmin del mes m
    tmin_mes = loadGridData(
        dataset = ruta_tmin,
        var = "tmin",
        latLim = lat,
        lonLim = lon,
        years = 1981:2021,
        season = m_num) %>% suppressMessages() %>% suppressWarnings()
    
    # Calcular tmean = (tmax + tmin) / 2
    tmean_mes = gridArithmetics(tmax_mes, tmin_mes, operator = "+")
    tmean_mes = gridArithmetics(tmean_mes, 2, operator = "/")

    # Guardar en un objeto con nombre dinámico
    assign(paste0("pti_tmean_", m_str), tmean_mes, envir = .GlobalEnv)
    assign(paste0("pti_tmin_", m_str), tmin_mes, envir = .GlobalEnv)
}

In [ ]:
## Cargo datos de temperatura media y mínima, humedad relativa y precipitación

# Rutas
ruta_hurs = "/lustre/gmeteo/PTICLIMA/DATA/OBSERVATIONS/PTI-grid-v0/data/Iberia/day/hr_daily_grid_pen.nc"
ruta_pr = "/lustre/gmeteo/PTICLIMA/DATA/OBSERVATIONS/PTI-grid-v0/data/Iberia/day/pr_daily_grid_pen.nc"

meses_num = 1:12
meses_str = sprintf("%02d", meses_num)

for (m in seq_along(meses_num)) {
    m_num = meses_num[m]
    m_str = meses_str[m]

    # Cargar hurs del mes m
    hurs_mes = loadGridData(
        dataset = ruta_hurs,
        var = "hr",
        latLim = lat,
        lonLim = lon,
        years = 1981:2021,
        season = m_num) %>% suppressMessages() %>% suppressWarnings()

    # Cargar pr del mes m
    pr_mes = loadGridData(
        dataset = ruta_pr,
        var = "pr",
        latLim = lat,
        lonLim = lon,
        years = 1981:2021,
        season = m_num) %>% suppressMessages() %>% suppressWarnings()
    
    assign(paste0("pti_hurs_", m_str), hurs_mes, envir = .GlobalEnv)
    assign(paste0("pti_pr_", m_str), pr_mes, envir = .GlobalEnv)
}

## 2. Carga datos ERA5-Land resolución (0.1º - 10 km)

In [ ]:
## Cargo datos de temperatura mínima

# Definir años
anios = 1981:2021

# Función de carga
cargar_dato = function(anio, mes) {
    yyyy   = paste0(anio)
    yyyymm = paste0(anio, mes)
    ruta = paste0("/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5-Land/data/Iberia/day/t2mn/", yyyy, "/",
                  "t2mn_ERA5-Land_", yyyymm, ".nc")
  
    data_aux = loadGridData(
        dataset = ruta,
        var     = "t2mn",
        lonLim  = lon,
        latLim  = lat) %>% suppressMessages %>% suppressWarnings
    return(data_aux)
}

# Bucle sobre los meses
for (mes in sprintf("%02d", 1:12)) {
    
    # Crear combinaciones solo para ese mes
    combinaciones = data.frame(
        anio = anios,
        mes  = mes,
        stringsAsFactors = FALSE)
  
    # Cargar datos de todos los años para ese mes
    datos_mes = lapply(1:nrow(combinaciones), function(i) {
        cargar_dato(combinaciones$anio[i], combinaciones$mes[i])})

    # Combinamos los grids en la dimensión temporal
    datos_mes = bindGrid(datos_mes, dimension = "time")

    # Pasamos las observaciones de Kelvin a Celsius
    datos_mes = gridArithmetics(datos_mes, 273.15, operator = "-")
    attr(datos_mes$Variable, "units") = "degC"
  
    # Guardar en un objeto con nombre dinámico
    assign(paste0("era5_t2mn_", mes), datos_mes, envir = .GlobalEnv)}

In [6]:
## Cargo datos de precipitación

# Función de carga
cargar_dato = function(anio, mes) {
    yyyy   = paste0(anio)
    yyyymm = paste0(anio, mes)
    ruta = paste0("/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5-Land/data/Iberia/day/tp/", yyyy, "/",
                  "tp_ERA5-Land_", yyyymm, ".nc")
  
    data_aux = loadGridData(
        dataset = ruta,
        var     = "tp",
        lonLim  = lon,
        latLim  = lat) %>% suppressMessages %>% suppressWarnings
    return(data_aux)
}

# Bucle sobre los meses
for (mes in sprintf("%02d", 1:12)) {
    
    # Crear combinaciones solo para ese mes
    combinaciones = data.frame(
        anio = anios,
        mes  = mes,
        stringsAsFactors = FALSE)
  
    # Cargar datos de todos los años para ese mes
    datos_mes = lapply(1:nrow(combinaciones), function(i) {
        cargar_dato(combinaciones$anio[i], combinaciones$mes[i])})

    # Combinamos los grids en la dimensión temporal
    datos_mes = bindGrid(datos_mes, dimension = "time")

    # Pasamos las observaciones de m a mm
    datos_mes = gridArithmetics(datos_mes, 1000, operator = "*")
    attr(datos_mes$Variable, "units") = "mm"
  
    # Guardar en un objeto con nombre dinámico
    assign(paste0("era5_pr_", mes), datos_mes, envir = .GlobalEnv)}

In [3]:
## Cargo datos de temperatura media

# Función de carga
cargar_dato = function(anio, mes) {
    yyyy   = paste0(anio)
    yyyymm = paste0(anio, mes)
    ruta = paste0("/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5-Land/data/Iberia/day/t2m/", yyyy, "/",
                  "t2m_ERA5-Land_", yyyymm, ".nc")
  
    data_aux = loadGridData(
        dataset = ruta,
        var     = "t2m",
        lonLim  = lon,
        latLim  = lat) %>% suppressMessages %>% suppressWarnings
    return(data_aux)
}

# Bucle sobre los meses
for (mes in sprintf("%02d", 1:12)) {
    
    # Crear combinaciones solo para ese mes
    combinaciones = data.frame(
        anio = anios,
        mes  = mes,
        stringsAsFactors = FALSE)
  
    # Cargar datos de todos los años para ese mes
    datos_mes = lapply(1:nrow(combinaciones), function(i) {
        cargar_dato(combinaciones$anio[i], combinaciones$mes[i])})

    # Combinamos los grids en la dimensión temporal
    datos_mes = bindGrid(datos_mes, dimension = "time")

    # Pasamos las observaciones de Kelvin a Celsius
    datos_mes = gridArithmetics(datos_mes, 273.15, operator = "-")
    attr(datos_mes$Variable, "units") = "degC"
  
    # Guardar en un objeto con nombre dinámico
    assign(paste0("era5_t2m_", mes), datos_mes, envir = .GlobalEnv)}

In [4]:
## Cargo datos de temperatura del punto de rocío

# Función de carga
cargar_dato = function(anio, mes) {
    yyyy   = paste0(anio)
    yyyymm = paste0(anio, mes)
    ruta = paste0("/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5-Land/data/Iberia/day/d2m/", yyyy, "/",
                  "d2m_ERA5-Land_", yyyymm, ".nc")
  
    data_aux = loadGridData(
        dataset = ruta,
        var     = "d2m",
        lonLim  = lon,
        latLim  = lat) %>% suppressMessages %>% suppressWarnings
    return(data_aux)
}

# Bucle sobre los meses
for (mes in sprintf("%02d", 1:12)) {
    
    # Crear combinaciones solo para ese mes
    combinaciones = data.frame(
        anio = anios,
        mes  = mes,
        stringsAsFactors = FALSE)
  
    # Cargar datos de todos los años para ese mes
    datos_mes = lapply(1:nrow(combinaciones), function(i) {
        cargar_dato(combinaciones$anio[i], combinaciones$mes[i])})

    # Combinamos los grids en la dimensión temporal
    datos_mes = bindGrid(datos_mes, dimension = "time")

    # Pasamos las observaciones de Kelvin a Celsius
    datos_mes = gridArithmetics(datos_mes, 273.15, operator = "-")
    attr(datos_mes$Variable, "units") = "degC"
  
    # Guardar en un objeto con nombre dinámico
    assign(paste0("era5_d2m_", mes), datos_mes, envir = .GlobalEnv)}

In [5]:
## Calculo la humedad relativa

for (m in sprintf("%02d", 1:12)) {
    cmd = paste0(
        "era5_hurs_", m, " = tdps2hurs(tdps = era5_d2m_", m,
        ", tas = era5_t2m_", m, ") %>% suppressMessages %>% suppressWarnings")
    eval(parse(text = cmd))
}

In [7]:
## Guardo datos

for (m in 1:12) {
    # Datos temperatura media
    saveRDS(get(paste0("era5_t2m_", sprintf("%02d", m))),
            paste0("Data_ERA5-Land/era5_tmean_", sprintf("%02d", m), ".rds"))

    # Datos humedad relativa
    saveRDS(get(paste0("era5_hurs_", sprintf("%02d", m))),
            paste0("Data_ERA5-Land/era5_hurs_", sprintf("%02d", m), ".rds"))
    
    # Datos temperatura mínima
    saveRDS(get(paste0("era5_t2mn_", sprintf("%02d", m))),
            paste0("Data_ERA5-Land/era5_tmin_", sprintf("%02d", m), ".rds"))

    # Datos precipitación
    saveRDS(get(paste0("era5_pr_", sprintf("%02d", m))),
            paste0("Data_ERA5-Land/era5_pr_", sprintf("%02d", m), ".rds"))
}

## 3. Interpolo PTI-grid-v0 (0.025º) a ERA5-land (0.1º)

In [ ]:
# Upscaling de la resolución de las observaciones a ERA5_Land

for (m in 1:12) {
    # Temperatura media
    assign(paste0("pti_tmean_ups_", sprintf("%02d", m)),
           interpGrid(get(paste0("pti_tmean_", sprintf("%02d", m))),
                      new.coordinates = getGrid(get(paste0("era5_t2m_", sprintf("%02d", m)))),
                      method = "bilinear") %>% suppressMessages %>% suppressWarnings)

    # Humedad relativa
    assign(paste0("pti_hurs_ups_", sprintf("%02d", m)),
           interpGrid(get(paste0("pti_hurs_", sprintf("%02d", m))),
                      new.coordinates = getGrid(get(paste0("era5_hurs_", sprintf("%02d", m)))),
                      method = "bilinear") %>% suppressMessages %>% suppressWarnings)

    # Temperatura mínima
    assign(paste0("pti_tmin_ups_", sprintf("%02d", m)),
           interpGrid(get(paste0("pti_tmin_", sprintf("%02d", m))),
                      new.coordinates = getGrid(get(paste0("era5_t2mn_", sprintf("%02d", m)))),
                      method = "bilinear") %>% suppressMessages %>% suppressWarnings)

    # Precipitación
    assign(paste0("pti_pr_ups_", sprintf("%02d", m)),
           interpGrid(get(paste0("pti_pr_", sprintf("%02d", m))),
                      new.coordinates = getGrid(get(paste0("era5_pr_", sprintf("%02d", m)))),
                      method = "bilinear") %>% suppressMessages %>% suppressWarnings)
}

In [ ]:
## Guardo datos

# Datos interpolados de PTI
for (m in 1:12) {
    # Guardar tmean
    saveRDS(get(paste0("pti_tmean_ups_", sprintf("%02d", m))),
            paste0("Data_PTI-v0/pti_tmean_", sprintf("%02d", m), ".rds"))
    
    # Guardar hurs
    saveRDS(get(paste0("pti_hurs_ups_", sprintf("%02d", m))),
            paste0("Data_PTI-v0/pti_hurs_", sprintf("%02d", m), ".rds"))

    # Guardar tmin
    saveRDS(get(paste0("pti_tmin_ups_", sprintf("%02d", m))),
            paste0("Data_PTI-v0/pti_tmin_", sprintf("%02d", m), ".rds"))

    # Guardar pr
    saveRDS(get(paste0("pti_pr_ups_", sprintf("%02d", m))),
            paste0("Data_PTI-v0/pti_pr_", sprintf("%02d", m), ".rds"))
  
}